 ## <center> Naive Bayes Classifier </center>

El teorema de Bayes es una ecuación que describe **la relación de probabilidades condicionales de cantidades estadísticas**.

En clasificación bayesiana estamos interesados en encontrar la probabilidad de que ocurra una “clase” dadas unas características observadas (datos). Lo podemos escribir como **P( Clase | Datos)**. El teorema de Bayes nos dice cómo lo podemos expresar en términos de cantidades que podemos calcular directamente:

<h1> 
    $$ \frac {P(Comprar|Datos)}
        {P(Alquilar|Datos)}
                = \frac{P(Datos|Comprar) * P(Comprar)}
                                     {P(Datos|Alquilar) * P(Alquilar)}$$</h1>

con esta maniobra, nos deshacemos del denominador de la ecuación anterior P(Datos) el llamado “probabilidad marginal”. 

<h1>
    $$posterior(comprar) 
                = \frac{P(comprar)p(lngresos|comprar)p(ahorros|comprar)p(hijos|comprar)}
                                        {probabilidadmarginal}$$</h1>

<h1>
    $$posterior(alquilar) 
                =\frac{P(alquilar)p(lngresos|alquilar)p(ahorros|alquilar)p(hijos|alquilar)}
                                        {probabilidad marginal}$$</h1>

Posterior de comprar es lo que queremos hallar: P(comprar|datos).

 ## <center> Ejercicio Machine Learning: Gaussian Naive Bayes </center>

## ¿Comprar o alquilar casa? ¿Qué me conviene?

En este ejercicio, usaremos el algoritmo de Gaussian Naive Bayes para decidir si nos conviene Alguilar o Comprar una casa 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sb

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest

## Importemos Datos de entrada

REFERENCIAS: 

Ingresos y gastos son mensuales de 1 personas o 2 si están casados.

Trabajo: 
0-sin trabajo 
1-autonomo, 
2-asalariado, 
3-empresario, 
4-Autonomos, 
5-Asalariados, 
6-Autonomo y Asalariado, 
7-Empresario y Autonomo, 
8-Empresarios o empresario y autónomo.

Estado_civil: 0-soltero 1-casado 2-divorciado

Hijos: Cantidad de hijos menores (no trabajan)

Comprar: 0-mejor alquilar 1-Comprar casa.

Hipoteca fija a 30 años con interes.

In [ ]:
dataframe = pd.read_csv(r"comprar_alquilar.csv")
dataframe.head(10)


*cantidad de muestras de comprar o alquilar tenemos:*

In [ ]:
print(dataframe.groupby('comprar').size())

*histograma de las características quitando la columna de resultados (comprar):*

In [ ]:
dataframe.drop(['comprar'], axis=1).hist()
plt.show()

# Preprocesamos los datos

Creamos 2 columnas nuevas. En una agrupamos:

-los gastos mensuales 

-el monto a financiar para comprar la casa

In [ ]:
dataframe['gastos']=(dataframe['gastos_comunes']+dataframe['gastos_otros']+dataframe['pago_coche'])
dataframe['financiar']=dataframe['vivienda']-dataframe['ahorros']
dataframe.drop(['gastos_comunes','gastos_otros','pago_coche'], axis=1).head(10)

### Información estadística de los datos

In [ ]:
reduced = dataframe.drop(['gastos_comunes','gastos_otros','pago_coche'], axis=1)
reduced.describe()

## Feature Selection
Selección de las características de entrada.

### Veamos la correlación entre columnas

In [ ]:
colormap = plt.cm.viridis
plt.figure(figsize=(12,12))
plt.title('Correlación de características de Pearson', y=1.05, size=15)
sb.heatmap(reduced.astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)

Usemos la clase SelectKBest de SkLearn para elegir las 5 mejores características a usar.

In [ ]:
X=dataframe.drop(['comprar'], axis=1)
y=dataframe['comprar']

best=SelectKBest(k=5)
X_new = best.fit_transform(X, y)
X_new.shape
selected = best.get_support(indices=True)
print(X.columns[selected])

Veamos la correlación entre las 5 mejores Features

In [ ]:
used_features =X.columns[selected]

colormap = plt.cm.viridis
plt.figure(figsize=(12,12))
plt.title('Correlación de características de Pearson', y=1.05, size=15)
sb.heatmap(dataframe[used_features].astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
sb.pairplot(dataframe, hue='comprar',size=4,vars=used_features,kind='scatter')

# Creamos el modelo de Gaussian Naive Bayes

Dividimos en un set de Entrenamiento y otro de Test con el 20% de las entradas

In [ ]:
# Split dataset in training and test datasets
X_train, X_test = train_test_split(dataframe, test_size=0.2, random_state=6) 
y_train =X_train["comprar"]
y_test = X_test["comprar"]

Entrenamos el modelo

In [ ]:
# Instantiate the classifier
gnb = GaussianNB()
# Train classifier
gnb.fit(
    X_train[used_features].values,
    y_train
)
y_pred = gnb.predict(X_test[used_features])

# Resultados

In [ ]:
print('Precisión en el set de Entrenamiento: {:.2f}'
     .format(gnb.score(X_train[used_features], y_train)))
print('Precisión en el set de Test: {:.2f}'
     .format(gnb.score(X_test[used_features], y_test))) 

In [ ]:
# Print Test results
print("Total de Muestras en Test: {}\nFallos: {}"
      .format(
          X_test.shape[0],
          (y_test != y_pred).sum()
))

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

## Nuevas Predicciones

Hagamos 2 pruebas a modo de ejemplo en las que el algoritmo nos recomienda Alquilar (0) y Comprar (1) en el 2do caso

+ En un caso será una familia sin hijos con 2000  de ingresos que quiere comprar una casa de 200000 y tiene sólo 5000 ahorrados.
+ El otro será una familia con 2 hijos con ingresos por 6000 al mes, 34000 en ahorros y consultan si comprar una casa de 320000.

In [ ]:
#                 ['ingresos', 'ahorros', 'hijos', 'trabajo', 'financiar']
print(gnb.predict([[2000,        5000,     0,       5,         200000],
                   [6000,        34000,    2,       5,         320000] ]))
#Resultado esperado 0-Alquilar, 1-Comprar casa

Los resultados son los esperados, en el primer caso, recomienda Alquilar (0) y en el segundo comprar la casa (1).

**EJERCICIO RESOLVER COMO EJEMPLO** 

+ Caso 1: una familia con **5** hijos con 10000  de ingresos que quiere comprar una casa de 500000 y tiene sólo 20000 ahorrados.
+ Caso 2: una familia con **1** hijo con ingresos de 8000 al mes, 60000 en ahorros y consultan si comprar una casa de 100000 
+ Caso 3: una familia **sin** hijo con ingresos de 15000 al mes, 40000 en ahorros y consultan si comprar una casa de 150000